In [1]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

In [3]:
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = user_secrets.get_secret("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = "langchain kaggle"
os.environ["MISTRAL_API_KEY"] = user_secrets.get_secret("mistral_legalBot")

In [4]:
!pip install -U langchain-mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 1.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.6.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompatible.
pandas-gbq 0.28.0 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2

In [6]:
from langchain.chat_models import init_chat_model

model = init_chat_model("mistral-large-latest", model_provider="mistralai")

In [12]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Determine the appropriate marks for the students answer"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", """To solve the problem do the following:
- First, work out your own solution to the problem including the final total. 
- Then compare your solution to the student's solution \ 
and evaluate if the student's solution is correct or not. 
Don't decide if the student's solution is correct until 
you have done the problem yourself.

Use the following format:
Question:
```
question here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Is the student's solution the same as actual solution \
just calculated:
```
yes or no
```
Student grade:
```
correct or incorrect
```

Question:
```
{question}
``` 
Student's solution:
```
{student}
```
Actual solution:""")])

In [13]:
question = """I'm building a solar power installation and I need help \
working out the financials. 
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost \
me a flat $100k per year, and an additional $10 / square \
foot
What is the total cost for the first year of operations \
as a function of the number of square feet."""

student = """Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000"""

In [14]:
prompt = prompt_template.invoke({"question": question, "student": student})

In [15]:
print(prompt)

messages=[SystemMessage(content='Determine the appropriate marks for the students answer', additional_kwargs={}, response_metadata={}), HumanMessage(content="To solve the problem do the following:\n- First, work out your own solution to the problem including the final total. \n- Then compare your solution to the student's solution \\ \nand evaluate if the student's solution is correct or not. \nDon't decide if the student's solution is correct until \nyou have done the problem yourself.\n\nUse the following format:\nQuestion:\n```\nquestion here\n```\nStudent's solution:\n```\nstudent's solution here\n```\nActual solution:\n```\nsteps to work out the solution and your solution here\n```\nIs the student's solution the same as actual solution just calculated:\n```\nyes or no\n```\nStudent grade:\n```\ncorrect or incorrect\n```\n\nQuestion:\n```\nI'm building a solar power installation and I need help working out the financials. \n- Land costs $100 / square foot\n- I can buy solar panels 

In [16]:
response = model.invoke(prompt)

In [18]:
print(response.content)

```
Let x be the size of the installation in square feet.
Costs:
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 10x
Total cost: 100x + 250x + 100,000 + 10x = 360x + 100,000
```
Is the student's solution the same as actual solution just calculated:
```
no
```
Student grade:
```
incorrect
```


# How about **Partial Marking**?

In [22]:
from langchain_core.prompts import ChatPromptTemplate


system_template = "Determine the appropriate marks for the student's answer based on the total marks provided."

prompt_template = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("user", """To solve the problem, do the following:
- First, work out your own solution to the problem, including the final total.
- Then compare your solution to the student's solution and evaluate if the student's solution is correct. If it is incorrect, determine how many marks should be awarded based on the type and severity of errors (e.g., minor calculation mistakes vs. conceptual errors).
- Assign a score to the student's solution out of the provided total marks.

Use the following format:
Question:
```
question here
```
Total marks:
```
Total marks here
```
Student's solution:
```
student's solution here
```
Actual solution:
```
steps to work out the solution and your solution here
```
Comparison of solutions:
```
Describe how the student's solution compares to your solution. Highlight any errors or differences.
```
Student marks:
```
X out of {total_marks} (explain the reasoning)
```
Question:
```
{question}
```
Total marks:
```
{total_marks}
```
Student's solution:
```
{student}
```
Actual solution:""")])

In [24]:
total_marks = 10

In [25]:
prompt = prompt_template.invoke({"question": question, "student": student, "total_marks":total_marks})

In [26]:
response = model.invoke(prompt)

In [27]:
print(response.content)

To determine the total cost for the first year of operations as a function of the number of square feet, we need to consider the costs of land, solar panels, and maintenance.

Let \( x \) be the size of the installation in square feet.

1. **Land cost**:
   \[
   \text{Land cost} = 100x
   \]

2. **Solar panel cost**:
   \[
   \text{Solar panel cost} = 250x
   \]

3. **Maintenance cost**:
   \[
   \text{Maintenance cost} = 100,000 + 10x
   \]

Adding these costs together, we get the total cost for the first year of operations:
\[
\text{Total cost} = 100x + 250x + 100,000 + 10x = 360x + 100,000
\]

Comparison of solutions:
```
The student's solution has a mistake in the maintenance cost calculation. The student incorrectly used $100x$ instead of $10x$ for the variable part of the maintenance cost. This error affects the total cost function.
```

Student marks:
```
7 out of 10

Reasoning:
- The student correctly identified and included the land cost and solar panel cost.
- The student co

# MD?

In [29]:
from IPython.display import Markdown, display

# Assuming `response.content` contains the model's response
response_content = response.content

# Display the content as Markdown
display(Markdown(response_content))

To determine the total cost for the first year of operations as a function of the number of square feet, we need to consider the costs of land, solar panels, and maintenance.

Let \( x \) be the size of the installation in square feet.

1. **Land cost**:
   \[
   \text{Land cost} = 100x
   \]

2. **Solar panel cost**:
   \[
   \text{Solar panel cost} = 250x
   \]

3. **Maintenance cost**:
   \[
   \text{Maintenance cost} = 100,000 + 10x
   \]

Adding these costs together, we get the total cost for the first year of operations:
\[
\text{Total cost} = 100x + 250x + 100,000 + 10x = 360x + 100,000
\]

Comparison of solutions:
```
The student's solution has a mistake in the maintenance cost calculation. The student incorrectly used $100x$ instead of $10x$ for the variable part of the maintenance cost. This error affects the total cost function.
```

Student marks:
```
7 out of 10

Reasoning:
- The student correctly identified and included the land cost and solar panel cost.
- The student correctly included the flat maintenance cost of $100,000.
- However, the student made a significant error in the variable part of the maintenance cost, using $100x$ instead of $10x$. This error affects the accuracy of the total cost function.
- Despite the error, the student demonstrated a good understanding of the problem and the correct approach to solving it.
```